## CS431/631 Big Data Infrastructure
### Winter 2022 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Clement Wu
* **ID:** 20775633

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



Run the next block to download the text file (`Shakespeare.txt`) and the Python tokenizer module (`simple_tokenize.py`).

In [ ]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of 
distinct token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$.

In [ ]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize

# create single token list
single = []

#create pairs of token list
pairs = []

# Now, let's tokenize Shakespeare's plays
with open('Shakespeare.txt') as f:
    for line in f:
        # tokenize, one line at a time
        t = simple_tokenize(line)

        # create temporary list of pairs of tokens in one line
        tmp = ['( ' + i + ' , ' + j + ' )' for i in t for j in t if i!=j]
        
        # update single token list
        single.extend(t)

        # update token pairs list
        pairs.extend(tmp)

# close the file
f.close()

# take distinct items in both lists
single = set(single)
pairs = set(pairs)

# number count of both lists
print("The number of distinct tokens: " + str(len(single)) + " , " + "The number of distinct tokens: "+ str(len(pairs)))




The number of distinct tokens: 25975 , The number of distinct tokens: 1969760


---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.


In [ ]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log

# the program gets the count of lines of all existing tokens and token pairs, also it calculates
# the total number of lines
def tokenCounter():
  # create single token dictionary
  single = {}

  # create pair token dictionary
  pair = {}

  # line counter
  lineNum = 0

  # Now, let's tokenize Shakespeare's plays
  with open('Shakespeare.txt') as f:
      for line in f:
          # update line number
          lineNum += 1

          # tokenize, one line at a time
          t = simple_tokenize(line)

          # create temporary list of pairs of tokens in one line
          tmp = ['( ' + i + ' , ' + j + ' )' for i in t for j in t if i!=j]
          
          # update single token dictionary
          for token in set(t):
            if token in single.keys():
              single[token] += 1
            else:
              single[token] = 1

          # update token pairs dictionary
          for pairToken in set(tmp):
            if pairToken in pair.keys():
              pair[pairToken] += 1
            else:
              pair[pairToken] = 1

  # close the file
  f.close()
  
  return single, pair, lineNum

# the program inputs a token, a dictionary, and total number of lines. It outputs
# the probility of token appearing in one line
def probability(token, dictionary, lineNum):
  return dictionary[token]/lineNum

# the program calculates the pmi value by taking two tokens, a single token dictionary and pair token dictionary
# and total number of lines
def pmi(x, y, single, pair, lineNum):
  return log(probability('( ' + x + ' , ' + y + ' )', pair, lineNum)/(probability(x, single, lineNum)*probability(y, single, lineNum)), 10)

# the program generetes top five tokens that paired with an input token, which have the highest
# pmi value
def top5pmi(x, single, pair, threshold, lineNum):
  # create y:pmi pairs and filter out items below threshold
  filtered = {k.split(" ")[3] : pmi(k.split(" ")[1], k.split(" ")[3], single, pair, lineNum) for k, v in pair.items() if v >= threshold and k.split(" ")[1] == x}

  # change dictionary to list of [y,pmi]
  filtered_list = list(map(list, filtered.items()))

  # sort list by pmi value
  sort_list = sorted(filtered_list, key = lambda item: item[1], reverse = True)

  # check of length is shorter than 5
  if len(sort_list) < 5:
    return sort_list
  return sort_list[:5]

# run the program to get all information
single, pair, lineNum = tokenCounter()

while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)

    # one-token queries
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue
        
        # check if x exists in the file
        if q_tokens[0] not in single.keys():
          print("token does not exist!")
          continue

        # get top 5 pmi value
        top5 = top5pmi(q_tokens[0], single, pair, threshold, lineNum)

        print("  n({0}) = {1}".format(q_tokens[0], single[q_tokens[0]]))
        print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0], threshold))
        for i in range(len(top5)):
          # print top 5 pmi values along with its x and y info
          print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(q_tokens[0], top5[i][0], pair['( ' + q_tokens[0] + ' , ' + top5[i][0] + ' )'], top5[i][1]))    

    # two-token queries
    elif len(q_tokens) == 2:
        if q_tokens[0] not in single.keys() or q_tokens[1] not in single.keys():
          print("at least one token does not exist!")
          continue

        # print x, y and their mutual frequency
        print("  n({0},{1}) = {2}".format(q_tokens[0], q_tokens[1], pair['( ' + q_tokens[0] + ' , ' + q_tokens[1] + ' )']))

        # print pmi value
        print("  PMI({0},{1}) = {2}".format(q_tokens[0], q_tokens[1], pmi(q_tokens[0],q_tokens[1], single, pair, lineNum)))
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")


Input 1 or 2 space-separated tokens (return to quit): man
Input a positive integer frequency threshold: 20
  n(man) = 1831
  high PMI tokens with respect to man (threshold: 20):
    n(man,honest) = 45,  PMI(man,honest) = 1.0282622233578977
    n(man,old) = 89,  PMI(man,old) = 0.9302116488011463
    n(man,every) = 56,  PMI(man,every) = 0.885957883693481
    n(man,any) = 73,  PMI(man,any) = 0.847262207567516
    n(man,woman) = 25,  PMI(man,woman) = 0.7060429286239783
Input 1 or 2 space-separated tokens (return to quit): 


---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

#### Answer to Question 3:

 My program has O(n^3) time complexity. If input file gets m times larger, the time required for the program to execute one iteration will gets m times larger, the time compexity of the program will grow m^3 higher. This means that my program will get slower and the time required to run it will grow exponentially. Since we are accumulating the stats of tokens, if ther file gets arebitrary large, it may cause memory overflow. This will make the program crash.






---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.